In [ ]:
# --------------------------------------
# 0️⃣ Setup environment and imports
# --------------------------------------
import sys
import os

# Add repo root to sys.path so 'src' can be imported
repo_root = os.path.abspath(os.getcwd())
if repo_root not in sys.path:
    sys.path.append(repo_root)

import pandas as pd
from src.data_loader import load_country_data
from src.cleaning import clean_dataset
from src.eda import run_full_eda_pipeline

# --------------------------------------
# 1️⃣ Configure dataset
# --------------------------------------
# Change these two variables per country
country_name = "benin"  # or "togo", "sierraleone"
filename = f"data/{benin}-dataset.csv"  # replace with actual CSV file

# --------------------------------------
# 2️⃣ Load dataset
# --------------------------------------
df = load_country_data("data/benin-malanville.csv")

# Quick summary
print(f"Dataset shape: {df.shape}")
print("Columns:", df.columns.tolist())

# --------------------------------------
# 3️⃣ Clean dataset
# --------------------------------------
df_clean = clean_dataset(df)

# Optional: quick summary of cleaned data
print(f"Cleaned dataset shape: {df_clean.shape}")

# --------------------------------------
# 4️⃣ Run full EDA
# --------------------------------------
df_cleaned_final = run_full_eda_pipeline(df_clean, country=country_name)

# --------------------------------------
# 5️⃣ Save cleaned dataset (optional)
# --------------------------------------
output_file = f"data/{country_name}_clean.csv"
df_cleaned_final.to_csv(output_file, index=False)
print(f"Cleaned dataset saved to: {output_file}")


ModuleNotFoundError: No module named 'src'